In [ ]:
import pandas as pd
from tkinter import Tk, filedialog
import os
import smtplib  
from email.message import EmailMessage
from getpass import getpass
# Função para enviar o e-mail

def enviar_email():
    # Dicionário de filiais e e-mails
    tabela_filiais = {
        "1": ("Matriz", ["email_matriz@empresa.com"]),
        "2": ("Juazeiro Norte", ["rafael.borges@sss.tec.br", "marilia.marinho@sss.tec.br"]),
        "3": ("Sobral", ["wagner.ripardo@sss.tec.br", "paulo.ricardo@sss.tec.br", "prince.alysson@sss.tec.br"]),
        "5": ("São Luís", ["thalis.sales@sss.tec.br", "clayssiane.amorin@sss.tec.br", "erinaldo.costa@sss.tec.br"]),
        "6": ("Teresina", ["gilmara.macedo@sss.tec.br", "conceição.almeida@sss.tec.br", "juan.costa@sss.tec.br"]),
        "7": ("Maracanaú", []),
        "9": ("Parnaíba", ["conceição.almeida@sss.tec.br", "gracilene.gomes@sss.tec.br", "pedro.herique@sss.tec.br"]),
        "10": ("Juazeiro Bahia", []),
        "11": ("Petrolina", [])
    }

    emails_fixos = [
        "meucartaonu01@gmail.com", "tamyres.mendes@sss.tec.br",
        "rennan.lucas@sss.tec.br", "edson.victor@sss.tec.br"
    ]
    emails_copia = [
        "airton.gomes@sss.tec.br", "jobson.aragao@sss.tec.br",
        "luis.felipe@sss.tec.br", "fabricio.morais@sss.tec.br"
    ]

    # Exibir filiais disponíveis
    print("Selecione a filial para envio do e-mail:")
    for key, value in tabela_filiais.items():
        print(f"{key}: {value[0]}")

    # Seleção da filial
    filial = input("Digite o número da filial: ")

    if filial not in tabela_filiais:
        print("Filial inválida.")
        return

    nome_filial, emails_filial = tabela_filiais[filial]
    destinatarios = emails_filial + emails_fixos
    destinatarios_copia = emails_copia

    # Assunto e corpo do e-mail
    assunto = f"Abastecimento - {nome_filial}"
    corpo_email = f"Segue em anexo demanda de abastecimento da filial {nome_filial}, gerente e setor de compras, verificar e sugerir."

    # Selecionar arquivos para anexar
    Tk().withdraw()  # Esconder janela principal do Tkinter
    arquivos_anexos = filedialog.askopenfilenames(title="Selecione os arquivos para anexar")

    if not arquivos_anexos:
        print("Nenhum arquivo selecionado. O e-mail não será enviado.")
        return

    # Configurações do servidor SMTP
    smtp_servidor = "smtp.gmail.com"
    smtp_porta = 587
    remetente = "yurif3035@gmail.com"
    senha = "mypk hiye obns pgea"  


    # Criar e-mail
    msg = EmailMessage()
    msg["Subject"] = assunto
    msg["From"] = remetente
    msg["To"] = ", ".join(destinatarios)
    msg["Cc"] = ", ".join(destinatarios_copia)
    msg.set_content(corpo_email)

    # Anexar arquivos
    for arquivo in arquivos_anexos:
        with open(arquivo, "rb") as f:
            msg.add_attachment(f.read(), maintype="application", subtype="octet-stream", filename=os.path.basename(arquivo))

    # Enviar e-mail
    try:
        with smtplib.SMTP(smtp_servidor, smtp_porta) as server:
            server.starttls()
            server.login(remetente, senha)
            server.send_message(msg)
        print("E-mail enviado com sucesso!")
    except Exception as e:
        print(f"Erro ao enviar e-mail: {e}")


# Dicionário de Filiais
    filiais = {
        1: "Matriz",
        2: "Juazeiro Norte",
        3: "Sobral",
        5: "São Luís",
        6: "Teresina",
        7: "Maracanaú",
        9: "Parnaíba",
        10: "Juazeiro Bahia",
        11: "Petrolina"
    }


# Função para selecionar arquivo
def selecionar_arquivo():
    Tk().withdraw()
    arquivo = filedialog.askopenfilename(filetypes=[("Arquivos Excel", "*.xlsx")])
    return arquivo

# Função para identificar colunas
def identificar_colunas(df):
    cod_produto = ["CODPROD"]
    qunt_sugestao = ["SUG MTZ", "SUG TZ", "SUG MAR"]

    col_cod_produto = next((col for col in df.columns if any(p in col.upper() for p in cod_produto)), None)
    col_qunt_sugestao = next((col for col in df.columns if any(p in col.upper() for p in qunt_sugestao)), None)

    # Se não encontrar, pedir input do usuário
    if not col_cod_produto:
        col_cod_produto = input("Digite o nome da coluna do código do produto: ")
    if not col_qunt_sugestao:
        col_qunt_sugestao = input("Digite o nome da coluna de quantidade do produto: ")

    return col_cod_produto, col_qunt_sugestao

# Função para ler e processar arquivo
def ler_arquivo(arquivo):
    df = pd.read_excel(arquivo)
    print("Colunas detectadas:", df.columns.tolist())

    cod_produto, qunt_sugestao = identificar_colunas(df)

    print(f"Coluna de Código do Produto: {cod_produto}")
    print(f"Coluna de Quantidade do Produto: {qunt_sugestao}")

    return df[[cod_produto, qunt_sugestao]]



# Solicita número da filial
def solicitar_filial():
    while True:
        try:
            num_filial = int(input("Digite o número da filial: "))
            if num_filial in filiais:
                return num_filial, filiais[num_filial]
            else:
                print("Número de filial inválido. Tente novamente.")
        except ValueError:
            print("Entrada inválida. Digite um número válido.")

# Segmentação dos pedidos
def segmentar_pedidos(df, num_filial, nome_filial, caminho_destino):
    df.insert(0, "Filial", num_filial)  # Adiciona a coluna da filial
    num_arquivos = (len(df) // 30) + (1 if len(df) % 30 > 0 else 0)  # Calcula quantos arquivos serão gerados

    for i in range(num_arquivos):
        df_segmento = df.iloc[i * 30:(i + 1) * 30]
        nome_arquivo = f"abastecimento_{nome_filial}_{i+1}.csv"
        caminho_arquivo = os.path.join(caminho_destino, nome_arquivo)
        df_segmento.to_csv(caminho_arquivo, index=False, sep=";")
        print(f"Arquivo salvo: {caminho_arquivo}")

# Seleção do diretório de salvamento
def selecionar_diretorio():
    Tk().withdraw()
    return filedialog.askdirectory()

    

def menu_principal():
    while True:
        print("\n======= SISTEMA DE ABASTECIMENTO =======")
        print("1 - Segmentar pedidos")
        print("2 - Enviar e-mail")
        print("3 - Sair")
        opcao = input("Escolha uma opção: ")

        if opcao == "1":
            arquivo_selecionado = selecionar_arquivo()
            if arquivo_selecionado:
                df_filtrado = ler_arquivo(arquivo_selecionado)
                num_filial, nome_filial = solicitar_filial()
                caminho_destino = selecionar_diretorio()
                
                if caminho_destino:
                    segmentar_pedidos(df_filtrado, num_filial, nome_filial, caminho_destino)
                else:
                    print("Nenhum diretório foi selecionado.")
            else:
                print("Nenhum arquivo foi selecionado.")
        
        elif opcao == "2":
            enviar_email()
        
        elif opcao == "3":
            print("Saindo do programa...")
            break
        
        else:
            print("Opção inválida! Tente novamente.")

# Iniciar programa
if __name__ == "__main__":
    menu_principal()
